### Telemetry Hello World

This is a very a brief introduction to Spark and Telemetry in Python. You should have a look at the [tutorial](https://gist.github.com/vitillo/25a20b7c8685c0c82422) in Scala and the associated [talk](http://www.slideshare.net/RobertoAgostinoVitil/spark-meets-telemetry) if you are interested to learn more about Spark.

In [ ]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from moztelemetry.dataset import Dataset

%matplotlib inline

### Basics

We will use the Dataset API to fetch data.  Documentation can be found at: https://python-moztelemetry.readthedocs.io/en/stable/api.html#dataset

The goal of this example is to plot the startup distribution for each OS. Let's see how many parallel workers we have at our disposal:

In [3]:
sc.defaultParallelism

32

We can look at the schema of the dataset we are interested in:

In [4]:
Dataset.from_source('telemetry').schema

[u'submissionDate',
 u'sourceName',
 u'sourceVersion',
 u'docType',
 u'appName',
 u'appUpdateChannel',
 u'appVersion',
 u'appBuildId']

Let's create a Dataset of Telemetry submissions for a given submission date:

In [5]:
pings_dataset = (
    Dataset.from_source('telemetry')
    .where(docType='main')
    .where(appBuildId='20180721100146')
    .where(appUpdateChannel="nightly")
)

Select only the properties we need and then take a 10% sample:

In [ ]:
pings = (
    pings_dataset
    .select(
        'clientId',
        GC_MARK_MS_content='payload.processes.content.histograms.CONTENT_PAINT_TIME.values',
        osName='environment.system.os.name',
        gfx='environment.system.gfx')
    .records(sc, sample=0.01)
)

This 'sampling' is based on s3 files and is highly
susceptible to skew. Use only for quicker performance
while prototyping.
fetching 16.60107MB in 5 files...


In [ ]:
pings = (
    pings_dataset
    .records(sc, sample=0.01)
)

Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f92caf03668> ignored
Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f9288bfb578> ignored
Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f9288bfb578> ignored
Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f9288bfb578> ignored
Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f92caf03668> ignored
Process Process-39:
Process Process-46:
Exception KeyboardInterrupt: KeyboardInterrupt() in <function <lambda> at 0x7f9288bfb578> ignored
Process Process-48:
Process Process-40:
Traceback (most recent call last):
Process Process-29:
Process Process-47:
Process Process-36:
Traceback (most recent call last):
Process Process-41:
Traceback (most recent call last):
  File "/mnt/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Process-38:
Process Process-33:
Traceback (most r

  File "/mnt/anaconda2/lib/python2.7/multiprocessing/queues.py", line 115, in get
  File "/mnt/anaconda2/lib/python2.7/site-packages/concurrent/futures/process.py", line 122, in _process_worker
  File "/mnt/anaconda2/lib/python2.7/multiprocessing/queues.py", line 115, in get
    self._target(*self._args, **self._kwargs)
    call_item = call_queue.get(block=True)
    call_item = call_queue.get(block=True)
  File "/mnt/anaconda2/lib/python2.7/site-packages/concurrent/futures/process.py", line 122, in _process_worker
    self._target(*self._args, **self._kwargs)
    self._rlock.acquire()
KeyboardInterrupt
  File "/mnt/anaconda2/lib/python2.7/multiprocessing/queues.py", line 117, in get
  File "/mnt/anaconda2/lib/python2.7/multiprocessing/queues.py", line 115, in get
    call_item = call_queue.get(block=True)
    self._rlock.acquire()
KeyboardInterrupt
    call_item = call_queue.get(block=True)
KeyboardInterrupt
    self._rlock.acquire()
    self._rlock.acquire()
    self._rlock.acquire()


Let's filter out submissions with an invalid startup time:

In [ ]:
pings.count()

In [ ]:
pings.take(2)

In [ ]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('gfx', None).get('adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter
            
    # Convert the version to a tuple of integers.
    #if 'driverVersion' in adapter:
    #    p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p

def filter_ping(p):
    return 'adapter' in p
rpings = pings.map(rewrite_ping).filter(filter_ping)
rpings = rpings.cache()
rpings.count()

In [ ]:
rpings.filter(lambda p: p['adapter']['GPUActive'] == True).take(1)

To prevent pseudoreplication, let's consider only a single submission for each client. As this step requires a distributed shuffle, it should always be run only after extracting the attributes of interest with *Dataset.select()*.

In [ ]:
subset = (
    rpings
    .map(lambda p: (p['clientId'], p))
    .reduceByKey(lambda p1, p2: p1)
    .map(lambda p: p[1])
)

Caching is fundamental as it allows for an iterative, real-time development workflow:

In [ ]:
cached = subset.cache()

How many pings are we looking at?

In [ ]:
rpings.count()

In [ ]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/nvidia.json'))
devices = {}
for gen in gpu_db['10de'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = chipset[0]

In [ ]:
nvidia = cached.filter(lambda p: p["adapter"]["vendorID"] == '0x10de')
nvidia.map(lambda p: "mobile" if devices[int(p['adapter']['deviceID'], 16)].endswith("M") else "desktop").countByValue()

In [ ]:
wrQualified = nvidia.filter(lambda p: p["gfx"]["features"]["wrQualified"]["status"] == "available" )
wrQualified.count()

In [ ]:
wrQualified.map(lambda p: p["gfx"]["features"]["compositor"]).countByValue()

In [ ]:
nvidia.take(1)

In [ ]:
mobile = nvidia.filter(lambda p: devices[int(p['adapter']['deviceID'], 16)].endswith("M"))
mobile.map(lambda p: len(p['adapters'])).countByValue()

In [ ]:
mobile.filter(lambda p: len(p['adapters'])> 1).take(1)

In [ ]:
desktop = nvidia.filter(lambda p: not devices[int(p['adapter']['deviceID'], 16)].endswith("M"))
desktop.map(lambda p: len(p['adapters'])).countByValue()

In [ ]:
desktop.filter(lambda p: len(p['adapters'])> 1).take(1)

Let's group the startup timings by OS:

In [ ]:
grouped = (
    cached
    .map(lambda p: (p['osName'], p['firstPaint']))
    .groupByKey()
    .collectAsMap()
)

And finally plot the data:

In [ ]:
frame = pd.DataFrame({x: np.log10(pd.Series(list(y))) for x, y in grouped.items()})
plt.figure(figsize=(17, 7))
frame.boxplot(return_type='axes')
plt.ylabel('log10(firstPaint)')
plt.show()

In [ ]:
plt.title('startup distribution for Windows')
plt.ylabel('count')
plt.xlabel('log10(firstPaint)')
frame['Windows_NT'].plot(kind='hist', bins=50, figsize=(14, 7))

### Histograms

Let's extract a histogram of GC_MARK_MS (time spent running JS garbage collection mark phase) from the submissions:

(see https://developer.mozilla.org/en-US/docs/Web/JavaScript/Memory_Management for more information)

In [ ]:
histograms = (
    pings_dataset
    .select(GC_MARK_MS_content='payload.processes.content.histograms.GC_MARK_MS.values',
            GC_MARK_MS_parent='payload.histograms.GC_MARK_MS.values')
    .records(sc, sample=0.05)
)

- `payload.histograms.GC_MARK_MS.values` is a path to the GC_MARK_MS values of the parent (main) process
- `payload.processes.content.histograms.GC_MARK_MS.values` is a path to the GC_MARK_MS values of the child processes

Let's aggregate the histogram over all submissions and plot it as a histogram.  Since the parent and child processes are recorded separately, we can create a histogram for each one and then add them together.

Each histogram is a pandas series where the index is the bucket and the value is the count.

In [ ]:
def aggregate_series(s1, s2):
    """Function to sum up series; if one is None, return other"""
    if s1 is None:
        return s2
    if s2 is None:
        return s1
    return s1.add(s2, fill_value=0)

aggregated_content = (
    histograms
    .map(lambda p: pd.Series(p['GC_MARK_MS_content']))
    .reduce(aggregate_series)
)
aggregated_content.index = [int(i) for i in aggregated_content.index]
aggregated_content = aggregated_content.sort_index()

aggregated_parent = (
    histograms
    .map(lambda p: pd.Series(p['GC_MARK_MS_parent']))
    .reduce(aggregate_series)
)
aggregated_parent.index = [int(i) for i in aggregated_parent.index]
aggregated_parent = aggregated_parent.sort_index()

In [ ]:
plt.title('GC_MARK_MS_content')
aggregated_content.plot(kind='bar', figsize=(15, 7))

In [ ]:
plt.title('GC_MARK_MS_parent')
aggregated_parent.plot(kind='bar', figsize=(15, 7))

We can also aggregate the values of the parent and children processes:

In [ ]:
plt.title('GC_MARK_MS')
(aggregated_content + aggregated_parent).plot(kind='bar', figsize=(15, 7))

Keyed histograms follow a similar pattern. To extract a keyed histogram for which we know the key/label we are interested in:

In [ ]:
keyed_hist = (
    pings_dataset
    .select(redirects='payload.keyedHistograms.NETWORK_HTTP_REDIRECT_TO_SCHEME.https.values')
    .records(sc, sample=0.05)
)

Add up the counts of every ping and plot it:

In [ ]:
aggregated = (
    keyed_hist
    .filter(lambda p: p['redirects'] is not None)
    .map(lambda p: pd.Series(p['redirects']))
    .reduce(lambda c1, c2: c1 + c2)
)

In [ ]:
aggregated.plot(kind='bar', figsize=(15, 7))